<a id='home'></a>

# PostgreSQL & Python

This notebook consists of core concepts in SQL and their impelementation on PostgreSQL database.
The information and exercises on this notebook is  based on  the material from Zetcode.com's tutorial and postgresql.org's documentation. The notebook covers a wide variety of concepts in SQL by creating trivial databases and it is created as a memo for myself.

Work in process...

## Table Of Contents

- [Create Role and Database](#1)

- [Version](#2)

- [Inserting Data](#3)

- [Retrieving Data](#4)

- [The Dictionary Cursor](#5)

- [Parameterized Queries](#6)

- [Metadata](#7)

- [Export and Import of Data](#8)

- [Auto Increment Primary Key in PostgreSQL](#9)

- [INSERT ... ON CONFLICT DO NOTHING/UPDATE ("UPSERT")](#10)

- [11](#11)


<a id='1'></a>

## Create Role and Database  

In [1]:
# Check if the PostgreSQL server is running 
!service postgresql status

● postgresql.service - PostgreSQL RDBMS
   Loaded: loaded (/lib/systemd/system/postgresql.service; enabled; vendor preset: enabled)
   Active: active (exited) since Cts 2017-02-04 15:41:58 +03; 2h 25min ago
  Process: 2322 ExecStart=/bin/true (code=exited, status=0/SUCCESS)
 Main PID: 2322 (code=exited, status=0/SUCCESS)
   CGroup: /system.slice/postgresql.service

Şub 04 15:41:58 samet-UX305FA systemd[1]: Starting PostgreSQL RDBMS...
Şub 04 15:41:58 samet-UX305FA systemd[1]: Started PostgreSQL RDBMS.


Connect as postgres user and check the users and databases

    $ sudo su - postgres
    
    $ \du

Create a new role in the PostgreSQL system with the ability to create new databases.

    $ sudo -u postgres createuser samet
    
Create a new database with the owner samet

    $ sudo -u postgres createdb testdb -O samet

In [2]:
# see all databases
!psql -l

                                  List of databases
   Name    |  Owner   | Encoding |   Collate   |    Ctype    |   Access privileges   
-----------+----------+----------+-------------+-------------+-----------------------
 itunes    | samet    | UTF8     | en_US.UTF-8 | en_US.UTF-8 | 
 postgres  | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | 
 template0 | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | =c/postgres          +
           |          |          |             |             | postgres=CTc/postgres
 template1 | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | =c/postgres          +
           |          |          |             |             | postgres=CTc/postgres
 testdb    | samet    | UTF8     | en_US.UTF-8 | en_US.UTF-8 | 
(5 rows)



[Home](#home)

<a id='2'></a>

## Version

The Python script below connects to the previosly created testdb database and executes an SQL statement which returns the version of the PostgreSQL database. 

In [3]:
import psycopg2
import sys

con = None 
# Initialize the con variable to None in order not to lead to an error in the finally clause.
try:    
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/')  
    # The connect() method creates a new database session and returns a connection object.  
    cur = con.cursor()
    # From the connection, get the cursor object. The cursor is used to traverse the records from the result set. 
    cur.execute('SELECT version()')  
    # Call the execute() method of the cursor and execute the SQL statement.       
    ver = cur.fetchone() 
    # Fetch the data and since only one record is retrieved, call the fetchone() method. 
    print ver    

except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
     
finally:    
    if con:
        con.close()

('PostgreSQL 9.5.5 on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 5.3.1-14ubuntu2) 5.3.1 20160413, 64-bit',)


[Home](#home)

<a id='3'></a>

## Inserting Data

Create a Cars table and insert several rows to it.

In [10]:
!psql -c 'DROP TABLE IF EXISTS Cars' testdb

DROP TABLE


In [12]:
con = None

try:    
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/')  
    cur = con.cursor()
    
    cur.execute("CREATE TABLE Cars(Id INTEGER PRIMARY KEY, Name VARCHAR(20), Price INT)")
    cur.execute("INSERT INTO Cars VALUES(1,'Audi',52642)")
    cur.execute("INSERT INTO Cars VALUES(2,'Mercedes',57127)")
    cur.execute("INSERT INTO Cars VALUES(3,'Skoda',9000)")
    cur.execute("INSERT INTO Cars VALUES(4,'Volvo',29000)")
    cur.execute("INSERT INTO Cars VALUES(5,'Bentley',350000)")
    cur.execute("INSERT INTO Cars VALUES(6,'Citroen',21000)")
    cur.execute("INSERT INTO Cars VALUES(7,'Hummer',41400)")
    cur.execute("INSERT INTO Cars VALUES(%s, %s, %s)",(8,'Volkswagen',21600))    
    # a different way to insert
    con.commit()
    
except psycopg2.DatabaseError, e:
    if con:
        con.rollback()
        # In case of an error, roll back any possible changes to the database table
    print 'Error %s' % e    
    sys.exit()

finally:    
    if con: con.close()

Error relation "cars" already exists



SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [14]:
# check the tables created in the database
!psql -c '\dt' testdb 
# describe the table
!psql -c '\d Cars' testdb

       List of relations
 Schema | Name | Type  | Owner 
--------+------+-------+-------
 public | cars | table | samet
(1 row)

            Table "public.cars"
 Column |         Type          | Modifiers 
--------+-----------------------+-----------
 id     | integer               | not null
 name   | character varying(20) | 
 price  | integer               | 
Indexes:
    "cars_pkey" PRIMARY KEY, btree (id)



In [15]:
# check the table
!psql -c 'SELECT * FROM Cars' testdb

 id |    name    | price  
----+------------+--------
  1 | Audi       |  52642
  2 | Mercedes   |  57127
  3 | Skoda      |   9000
  4 | Volvo      |  29000
  5 | Bentley    | 350000
  6 | Citroen    |  21000
  7 | Hummer     |  41400
  8 | Volkswagen |  21600
(8 rows)



Create the same table but this time using the convenience executemany() method. 

In [16]:
cars = (
    (1, 'Audi', 52642),
    (2, 'Mercedes', 57127),
    (3, 'Skoda', 9000),
    (4, 'Volvo', 29000),
    (5, 'Bentley', 350000),
    (6, 'Citroen', 21000),
    (7, 'Hummer', 41400),
    (8, 'Volkswagen', 21600))

con = None

try:    
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/')  
    cur = con.cursor()
    
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INT PRIMARY KEY, Name TEXT, Price INT)")
    query = "INSERT INTO Cars (Id, Name, Price) VALUES (%s, %s, %s)"
    cur.executemany(query, cars)
    # The first parameter of this method is a parameterized SQL statement. 
    # The second parameter is the data, in the form of tuple of tuples.
    con.commit()
    
except psycopg2.DatabaseError, e:    
    if con:
        con.rollback()    
    print 'Error %s' % e    
    sys.exit(1)        
finally:   
    if con: con.close()

Check https://nelsonslog.wordpress.com/2015/04/27/inserting-lots-of-data-into-a-remote-postgres-efficiently/ for a faster implementation of executemany.

In [17]:
# check the table
!psql -c 'SELECT * FROM Cars' testdb

 id |    name    | price  
----+------------+--------
  1 | Audi       |  52642
  2 | Mercedes   |  57127
  3 | Skoda      |   9000
  4 | Volvo      |  29000
  5 | Bentley    | 350000
  6 | Citroen    |  21000
  7 | Hummer     |  41400
  8 | Volkswagen |  21600
(8 rows)



[Home](#home)

<a id='4'></a>

## Retrieving Data

After inserting some data into the database, lets retrieve it back. 

In [18]:
con = None

try:    
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/')  
    cur = con.cursor()
    
    cur.execute("SELECT * FROM Cars")
    
    rows = cur.fetchall()
    print rows
    # returns list of tuples
    print 
    for row in rows:
        print row
        
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)    
finally:    
    if con: con.close()

[(1, 'Audi', 52642), (2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000), (5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600)]

(1, 'Audi', 52642)
(2, 'Mercedes', 57127)
(3, 'Skoda', 9000)
(4, 'Volvo', 29000)
(5, 'Bentley', 350000)
(6, 'Citroen', 21000)
(7, 'Hummer', 41400)
(8, 'Volkswagen', 21600)


Connect to the database and fetch the rows of the Cars table one by one. 

In [19]:
con = None

try:    
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/')  
    cur = con.cursor()
    
    cur.execute("SELECT * FROM Cars")

    while True:      
        row = cur.fetchone()        
        if row == None:
            break            
        print row[0], row[1], row[2]
        
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
finally:    
    if con: con.close()


1 Audi 52642
2 Mercedes 57127
3 Skoda 9000
4 Volvo 29000
5 Bentley 350000
6 Citroen 21000
7 Hummer 41400
8 Volkswagen 21600


[Home](#home)

<a id='5'></a>

#### The Dictionary Cursor

The default cursor returns the data in a list of tuples. The dict cursors allow to access to the retrieved records using an interface similar to the Python dictionaries instead of the tuples. This way the data can be refered by their column names. 

In [20]:
import psycopg2.extras

con = None

try:    
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/')  
    cursor = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
    cursor.execute("SELECT * FROM Cars")
    
    rows = cursor.fetchall()

    print rows
    print 
    for row in rows:
        print "%s %s %s" % (row["id"], row["name"], row["price"])
        
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)        
finally:    
    if con: con.close()

[[1, 'Audi', 52642], [2, 'Mercedes', 57127], [3, 'Skoda', 9000], [4, 'Volvo', 29000], [5, 'Bentley', 350000], [6, 'Citroen', 21000], [7, 'Hummer', 41400], [8, 'Volkswagen', 21600]]

1 Audi 52642
2 Mercedes 57127
3 Skoda 9000
4 Volvo 29000
5 Bentley 350000
6 Citroen 21000
7 Hummer 41400
8 Volkswagen 21600


[Home](#home)

<a id='6'></a>

#### Parameterized Queries

When parameterized queries are used, placeholders are used instead of directly writing the values into the statements. Parameterized queries increase security and performance. 

In [ ]:
con = None 

uId = 1
uPrice = 62300 

try:     
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/')   
    
    cur = con.cursor()

    cur.execute("UPDATE Cars SET Price={} WHERE Id={}".format(uPrice, uId))        
    con.commit()
    
    print "Number of rows updated: {}".format(cur.rowcount)
    
except psycopg2.DatabaseError, e:    
    if con:
        con.rollback()    
    print 'Error %s' % e    
    sys.exit(1)
finally:    
    if con: con.close()

In [ ]:
!psql -c 'SELECT * FROM cars WHERE id=1' testdb

In [ ]:
con = None

uid = 3

try:    
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/') 
    
    cur = con.cursor()

    cur.execute("SELECT * FROM Cars WHERE Id=%(id)s", {'id': uid } )
    # a different kind of place holder
    print cur.fetchone()

except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)  
finally:  
    if con: con.close()

[Home](#home)

<a id='7'></a>

## Metadata

Metadata is information about the data in the database. Metadata in a PostgreSQL database contains information about the tables and columns, in which the data is stored. Number of rows affected by an SQL statement is a metadata. Number of rows and columns returned in a result set belong to metadata as well.

Metadata in PostgreSQL can be obtained using from the description property of the cursor object or from the information_schema table.

Print the contents of the Cars table to the console and include the names of the columns too. The records are aligned with the column names. 

In [ ]:
con = None

try:
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/')   
    
    cur = con.cursor()
    
    cur.execute('SELECT * FROM Cars')
    
    col_names = [cn[0] for cn in cur.description]
    
    rows = cur.fetchall()
    print "%s %-10s %s" % (col_names[0], col_names[1], col_names[2])

    for row in rows:    
        print "%2s %-10s %s" % row
    
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
finally:    
    if con: con.close()

In the following example all the tables in the testdb database are displayed. 

In [ ]:
con = None

try:
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/')   
    cur = con.cursor() 
    cur.execute("""SELECT table_name FROM information_schema.tables 
       WHERE table_schema = 'public'""")    
        
    rows = cur.fetchall()

    for row in rows:
        print row[0]
        
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)    
finally:
    if con: con.close()

[Home](#home)

<a id='8'></a>

## Export and Import of Data

Export and import data by using copy_to() and copy_from() methods.In the below example, the data is copied from the Cars table into the cars file. 

In [ ]:
con = None
fout = None

try:
    con = psycopg2.connect(database='testdb', user='samet' , host='/var/run/postgresql/')  
    
    cur = con.cursor()
    fout = open('cars', 'w')
    cur.copy_to(fout, 'cars', sep="|")
    
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
except IOError, e:    
    print 'Error %s' % e   
    sys.exit(1)    
finally:
    if con: con.close()
    if fout: fout.close() 

In [ ]:
!cat cars

Now perform a reverse operation and import the dumped table back into the database table. First, delete the data from the Cars table. 

In [ ]:
! psql -c 'DELETE FROM Cars' testdb

Read the contents of the cars file and copy it back to the cars table. 

In [ ]:
con = None
f = None

try:   
    con = psycopg2.connect(database='testdb', user='samet', host='/var/run/postgresql/') 
    
    cur = con.cursor()
    f = open('cars', 'r')
    cur.copy_from(f, 'cars', sep="|")                    
    con.commit()
    
except psycopg2.DatabaseError, e:    
    if con:
        con.rollback()
    print 'Error %s' % e    
    sys.exit(1)
except IOError, e:    
    if con:
        con.rollback()
    print 'Error %s' % e   
    sys.exit(1)
    
finally:
    if con: con.close()
    if f: f.close()

The output shows that the saved Cars table is successfully recreated. 

In [ ]:
! psql -c 'SELECT * FROM Cars;' testdb

[Home](#home)

<a id='9'></a>

## Auto Increment Primary Key in PostgreSQL

    CREATE TABLE IF NOT EXISTS category (
    id SERIAL PRIMARY KEY,
    name varchar(30) DEFAULT NULL
    );

If you want to create the sequence yourself then you want to make the default value of id the next value in the sequence and that means saying default nextval('your_seq'):

    CREATE SEQUENCE your_seq;
    CREATE TABLE IF NOT EXISTS category (
      id integer PRIMARY KEY default nextval('your_seq'),
      name varchar(30) DEFAULT NULL
    );

To simulate the usual serial behavior you'll also want to make the sequence owned by the table:

alter sequence your_seq owned by category.id;


    CREATE SEQUENCE user_id_seq;
    CREATE TABLE user (
        user_id smallint NOT NULL DEFAULT nextval('user_id_seq')
    );
    ALTER SEQUENCE user_id_seq OWNED BY user.user_id;
    
-https://chartio.com/resources/tutorials/how-to-define-an-auto-increment-primary-key-in-postgresql/

[Home](#home)

<a id='10'></a>

## INSERT ... ON CONFLICT DO NOTHING/UPDATE ("UPSERT")


9.5 brings support for "UPSERT" operations.

INSERT is extended to accept an ON CONFLICT DO UPDATE/IGNORE clause. This clause specifies an alternative action to take in the event of a would-be duplicate violation.

For example, if we have a simple table with user accounts logins where we wanted to track the number of times that user had logged in:

      SELECT username, logins FROM user_logins;
      username | logins 
     ----------+--------
      James    |      4
      Lois     |      2
     (2 rows)

And we wanted to add 2 new logins, normally we'd have a problem if the primary key (or unique constraint) was violated:

       INSERT INTO user_logins (username, logins)
       VALUES ('Naomi',1),('James',1);
     ERROR:  duplicate key value violates unique constraint "users_pkey"
     DETAIL:  Key (username)=(James) already exists.

Unlike approaches using a Common Table Expression, the new command has no race conditions, guaranteeing either an insert or an update (provided there is no incidental error). ON CONFLICT DO UPDATE safely guarantees "insert-or-update" semantics, with no risk of the statement failing to perform one of those two actions for each row proposed for insertion (unless there was an independent error). Unlike SQL MERGE as implemented in other database systems, there is no implementation or semantic restriction that burdens users with having to consider that their UPSERT might fail due to a concurrent operation. In short, the new "UPSERT" feature is designed to be both as simple and as safe as possible.

Further example of new syntax:

     INSERT INTO user_logins (username, logins)
     VALUES ('Naomi',1),('James',1)
     ON CONFLICT (username)
     DO UPDATE SET logins = user_logins.logins + EXCLUDED.logins;
or 

     INSERT INTO user_logins (username, logins)
     VALUES ('Naomi',1),('James',1)
     ON CONFLICT (username)
     DO NOTHING;
     
https://wiki.postgresql.org/wiki/What%27s_new_in_PostgreSQL_9.5#INSERT_..._ON_CONFLICT_DO_NOTHING.2FUPDATE_.28.22UPSERT.22.29

[Home](#home)

<a id='11'></a>

####

[Home](#home)

<a id='7'></a>

####

[Home](#home)

<a id='7'></a>

####

[Home](#home)

#### References
-http://zetcode.com/db/postgresqlpythontutorial/

-http://dba.stackexchange.com/questions/4286/list-the-database-privileges-using-psql

-https://www.postgresql.org/docs/9.5/static/index.html

-https://chartio.com/resources/tutorials/how-to-define-an-auto-increment-primary-key-in-postgresql/

http://dbeaver.jkiss.org/ (client)

https://wiki.postgresql.org/wiki/What%27s_new_in_PostgreSQL_9.5#INSERT_..._ON_CONFLICT_DO_NOTHING.2FUPDATE_.28.22UPSERT.22.29